Imports

In [50]:
from time import sleep
import pyautogui, json

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager


Webdriver Initialization

In [59]:
url='https://villageinfo.in'

options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), options=options
)
pyautogui.hotkey("win", "d")
pyautogui.hotkey("alt", "tab")
pyautogui.hotkey("alt", "tab")

driver.get(url)

Collect Village Link List

In [60]:
driver.get(url)

village_links = []
state_elements = driver.find_elements(by=By.CSS_SELECTOR, value=".tab span a")

state_links = []
for state_element in state_elements:
    state_link = state_element.get_attribute("href")
    state_links.append(state_link)
    break

for state_link in state_links:
    driver.get(state_link)
    district_elements = driver.find_elements(
        by=By.CSS_SELECTOR, value=".vict tbody tr td a"
    )

    district_links = []
    for district_element in district_elements:
        district_link = district_element.get_attribute("href")
        district_links.append(district_link)

    for district_link in district_links:
        driver.get(district_link)
        tehsil_elements = driver.find_elements(
            by=By.CSS_SELECTOR, value=".vict tbody tr td a"
        )

        tehsil_links = []
        for tehsil_element in tehsil_elements:
            tehsil_link = tehsil_element.get_attribute("href")
            tehsil_links.append(tehsil_link)

        for tehsil_link in tehsil_links:
            driver.get(tehsil_link)
            village_elements = driver.find_elements(
                by=By.CSS_SELECTOR, value=".vict tbody tr td a"
            )

            for village_element in village_elements:
                village_link = village_element.get_attribute("href")
                village_links.append(village_link)


Collect Data

Dump Data to JSON

In [ ]:
with open("data.json", "w") as file1:
    json.dump(data, file1)

Quit Webdriver

In [46]:
driver.close()